## CNN

CNN has the same function as ordinary Neural network do. e.g.
- have same process of dot product
- have same differentiable score function
- same loss function(softmax, SVM)

**Q**: What changes and why CNN?

 **Reason**
   - **Reduce number of params**: In oridinary NN a single neuron of a layer is fully connected to all the neurons of the previous layers. In case of CIFAR-10, a single image has the shape[32,32,3] which means that a single neuron in the first layer needs 32*32*3=3072 params and a bias. since in nn the neurons don't share weights among them, the number of params for a single layer is then $s_j * 3073$, where $s_j$ means number of neurons. this semms a lot of params. moreover, We work with large imae than this. It's both wasteful and soon lead to overfitting.
   
  **Changes**: 
  - Since the input is image, they constrain the architecture based on their properties.
    - CNN neruons are 3 dimensional (witdth,height,depth). Images are input volume activations.
    - neurons are connected to only to a local region of the input volume(image)
    - final output layer (in case of cifar-10) is [1,1,10]. because the output volume of the last layer will be a vector of class scores. here, width=1,height=1 . so we will arrange the output vector along the depth dimension

     ![alt txt](images/cnn.PNG "cnn diagram")

## Layers of CNN

Three main types of layers.
- Convolutional Layer
- Pooling layer
- Fully-Connected Layer

Like ordinary NN we stack these layers to form a full ConvNet architecture. It's very important to understand the layers to work with CNN. Let's assume a CNN model for cifar-10 dataset.
- Input is a 3 dimensional image [32,32,3]
- CONV layer which takes the image as input: computes outputs of each neuron by dot product between the weights(size=keranlsize/filter size) and the local region on the image which is connected to filter. Final size [32,32,12] if we use 12 filters.
- ReLU applies elementwise activation function, such as max(0,x). **this leaves size of the volume unchanged**[32,32,12]
- POOL: this layer downsamples the volume spatially.
- FC: Although there can be more than one Fully-connected layer, the last one will compute the class scores. As the naem suggests the neurons of FC is connected to all the previous neurons. In case of cifar-10 image the result volume of the last FC layer will be 1X1X10. Here notice **class scores are along depth dimension**


Param(s) of layers:
- Convolutional and FC: have params { weights and biases}
- Pool and Relu: No params because they perform on fixed function

![alt txt](images/cnn1.PNG "cnn layers")
The activations of an example ConvNet architecture. The initial volume stores the raw image pixels (left) and the last volume stores the class scores (right). Each volume of activations along the processing path is shown as a column. Since it's difficult to visualize 3D volumes, we lay out each volume's slices in rows.The last layer volume holds the scores for each class, but here we only visualize the sorted top 5 scores.

## Convolutional Layer
Convolutional layers are the core building block of CNN. So it's important to understand the deep of it.
- **params**: Params of Conv layer is one or more 3 dimensional filters. e.g. a filter of size [5,5,3]. Look, the filters are small spatially but extends through the full depth of the image. During forward pass we convolve the filter spatially of the image but in full depth and compute **dot products between the entries of the filter and the input** at any position. We know dot product gives us a single scalar value. Thus after convolving throughtout the image we get a single scalar value for each convolution to a specific local region. Thus finally we end up with only a two dimensional feature map which contains the scalar values spatially. Now if we use more than one filter, we will get more activation map like this. This activations maps are kept along depth dimension. Thus the output of the convolution layer will be
[X,X,number_of_filters]. That means the depth is equal to the number of filters. I put X to the width and height position which will be discussed later. The early layers learns simple features and the later complex gradually.


**Local Connectivity**
- the filters are connected to only a local region of the input discussed earlier. This spatial extent of connectivity is called **receptive field**. But the extent of connectivity along the depth is full. In other words, connections are local in 2D and full along depth.

Example 1. For example, suppose that the input volume has size [32x32x3], (e.g. an RGB CIFAR-10 image). If the receptive field (or the filter size) is 5x5, then each neuron in the Conv Layer will have weights to a [5x5x3] region in the input volume, for a total of 5*5*3 = 75 weights (and +1 bias parameter). Notice that the extent of the connectivity along the depth axis must be 3, since this is the depth of the input volume.

Example 2. Suppose an input volume had size [16x16x20]. Then using an example receptive field size of 3x3, every neuron in the Conv Layer would now have a total of 3*3*20 = 180 connections to the input volume. Notice that, again, the connectivity is local in 2D space (e.g. 3x3), but full along the input depth (20).

![alt txt](images/cnn2.PNG "cnn")


***Spatial arrangement**
the number of neurons in the output volume of a conv layer depends on three hyperparameters.
- depth : number of filters
- stride: when stride 1 we move the filter one pixel at a time
- padding: used to preserve the spatial size of the input volume

If input size $W$ and filter size $F$ and padding $P$ then the number of output neurons is $(W-F+2P)/S +1$

For example for a 7x7 input and a 3x3 filter with stride 1 and pad 0 we would get a 5x5 output. With stride 2 we would get a 3x3 output. Lets also see one more graphical example:
![alt txt](images/cnn3.PNG "cnn")


**use of zero padding**:
If we use $P=(F-1)/2$ when stride $S$ is 1, then the spatial size of input and output volume will be same.

**parameter sharing**
If output size of a Conv layer is 55X55X96 then total neurons =290400. Each has 11X11X3=363 weights and a bias. total 290400 * 364 = 105,705,600 parameters. Assuming total 96 depth slices we can use 96 different set of weights and 96 bias.
For backward propagation, the gradients accross the depth slice will be added and only update a single set of weights per slice.

**Numpy Examples** :
- A depth column (or a fibre) at position (x,y) would be the activations X[x,y,:].
- A depth slice, or equivalently an activation map at depth d would be the activations X[:,:,d].
Example: suppose, input size is X.shape: (11,11,4), P=0,S=2, F=5. Therefore the output size will be (11-5)/2 +1= 4. Spatial size of each activation map is 4X4 and depth is 4. Activation map in the output volume V,
- V[0,0,0]=np.sum(X[:5,:5,:]*W0)+b0
  - spatial position 0,0 and depth position 0 means the first activation map
- V[1,0,0]=np.sum(X[2:7,:5,:]*W0)+b0
- V[2,0,0]=np.sum(X[4:9,:5,:]*W0)+b0
- V[3,0,0]=np.sum(X[6:11,:5,:]*W0)+b0

for second activation map, let's see some calculations,
- v[0,0,1]=np.sum(X[:5,:5,:]*W1)+b1
- v[0,1,1]=np.sum(X[:5,2:7,:]*W1)+b1
- v[2,3,1]=np.sum(X[4:9,6:11,:]*W1)+b1



### Summary
- Accepts a volume of size W1,H1,D1
- Requires hyeperparameters,
  - Number of filters K
  - spatial extent of filters F
  - stride S,
  - Padding P
- Produces output volume of size W2,H2,D2. Where,
  - W2=(W1-F+2P)/S +1
  - H2= (H1-F+2P)/S +1
  - D2=K
- total weights per filter is (F.F.D1) and a total of (F.F.D1)*K weights and K biases
**common setting**
- F=3, S=1, P=1


### Implementation of Conv Layer as matrix multiplication
Suppose we have input volume of size[227X227X3], filter size 11X11X3 , stride S=4, padding
- Local rregions in the input images are stretched out in an operation called im2col. For our above example, loacal region size is of size 11X11X3=363.we keep it as columns. having stride S=4 we will have then total (227-11)/4 +1=55 locations along width and height. Therefore the output matrix X_col with all the columns at different position in the input image will be of shape W_row=[363X3025].
- the weights of CONV layer are similarly stretched out into rows. For 96 filters, the shape of weight matrix will be [96X363]
- Therefore the result of the convolution is now simply equivalent to np.dot(W_row,X_col)
- Finally we need to reshape the result into 55X55x64


### Dilated Convolutions
So far we have only discussed CONV filters that are contiguous. However it's possible to have filters having space beteween each cell, called dilation. if dilation 0, then w[0]*x[0]+w[1]*x[1]+w[2]*x[2]. If we use dilation 1 then this would be  w[0]*x[0]+w[1]*x[2]+w[2]*x[4]

### Pooling Layer
It is common to add a pooling layer in between successive Conv layer. It reduces the spatial size of the input volume and resize it
**Reasons**
- To reduce parameters which refers to recover overfitting
- To reduce calculations
Output volumes:
- Accepts a volume of size W1,H1,D1
- Requires **two** hyeperparameters,
  - spatial extent of filters F
  - stride S,
- Produces output volume of size W2,H2,D2. Where,
  - W2=(W1-F)/S +1
  - H2= (H1-F)/S +1
  - D2=D1
- total weights per filter is (F.F.D1) and a total of (F.F.D1)*K weights and K biases
**common setting**
- F=3, S=1, P=1

**General Pooling**
- F=3 , S=2
- F=2, S=2

**Pooling types**
different typyes of pooling. e.g.
- max pooling
- average pooling
- L2 norm pooling
average pooling is historically used but recently has fallen out of favor. Max pooling is widely used nowadays

![alt txt](images/maxpool.PNG "max pooling")

**Backpropagation** 

Backpropagation for max(x,y) function is to only routing the gradient to the max values. Therefore it's common to keep track of the index of max activations.

**Getting rid of Pooling** 

Many people dislikes pooling. Instead they like to use larger stride once in a while to reduce the spatial extent of the input volume. Moreover, it's good to discard pooling layer in case of variational auto encoders(VAEs) or generative adversial networks (GANs).

### Fully Connected Layers
each neuron is connected to all the neurons of the previous layer
#### Converting FC layers to CONV layers
Although the connection type of FC layer and CONV layer is different their functional form is identical which is dot product. Therefore, it is possible to convert between FC and CONV layers.

**CONV to FC**: for any CONV layer there is a FC layer that outputs the same. In that case most of the values of weight matrix is zero except for a certain block(due to lacal connectivity)

**FC to CONV** 

Suppose a ConvNet architecture that takes input of size 224X224X3, then uses a series of pooling layer and finally have the size 7X7X512. After this we can use a fully connected layer of size 4096 and finally the last FC layer with 1000 neurons that compute the class scores. We can convert each of these FC into CONV layer.

- Replace the first FC layer that looks at 7X7X512 volume with a CONV layer that uses filter size F=7 , giving output volume [1X1x4096]
- Replace the second FC layer with a Conv layer that uses F=1, giving output volume[1X1X4096]
- Replace the last FC layer with a CONV layer with a CONv layer that uses F=1 , giving output [1X1X1000]

This conversion allows us to slide the original ConvNet very efficiently across many spatial positions in a larger image, in a single forward pass.

For example, if 224x224 image gives a volume of size [7x7x512] - i.e. a reduction by 32, then forwarding an image of size 384x384 through the converted architecture would give the equivalent volume in size [12x12x512], since 384/32 = 12. Following through with the next 3 CONV layers that we just converted from FC layers would now give the final volume of size [6x6x1000], since (12 - 7)/1 + 1 = 6. Note that instead of a single vector of class scores of size [1x1x1000], we’re now getting an entire 6x6 array of class scores across the 384x384 image.

### ConvNet Architectures
The most common ConvNet architecture follows the pattern,

INPUT->[[CONV-> RELU]*N -> POOL]*M -> [FC -> RELU]*K ->FC

Usually 0<=N<=3, M>=0, k>=0 and usually K<3


**Q** CONV layer size : small or larger? which is good.

**Answer** stacking layers of smaller filter is more efficient than to stack a larger layer

**Reason**

- **Learning Complex features**:

if we stack three 3X3 conv layers then the first layer will have 3X3 view of the input image, the second layer will have 3X3 view of first layer but 5X5 view of the input. similarly the third layer will have 7X7 view of the input image. Moreover there is nonlinearity function between layers. Therefore the architecture will learn more complex feature than a model where we only use a single CONV layer having filter size 7X7.

- **Reducing parameters**

if all the volume has C channels then the single Conv layer having filter size 7X7 will have C(7X7XC)=49C^2. But for stacked layers it will have 3X(CX(3X3XC))=27C^2


### Layer sizing patterns

- **Input layer** : should be divisible by two many times. Common numbers including 32( e.g CIFAR-10),64,96 e.t.c
- **CONV layer**: should be using small filters (e.g. 3X3 or at most 5X5),stride S=1, padding the input volume with zeros in such a way so that input and output spatial size remains the same. Generally we will use padding P=(F-1)/2 for this purpose.
- **Pooling layer** : Usually use 2X2 filter size with stride 2 Or 3X3 filter size with stride 2. No larger filter size than 3X3. Because in that case the pooling will be too lossy and agressive.


# CNN architecutres

- **LeNet**: First Successful Convolutional Neural Network used for reading zipcode.
- **AlexNet**: First work that popularized Convolutional Networks in Computer Vision. Used in ImageNet Challenge.
- **ZF Net** Modified version of Alex Net.
- **GoogLeNet**: Modified version of AlexNet. It reduces the number of parameters significantly to 4M from 60 M.
- **VGG Net**: Modified Version of AlexNet. It shows that the depth of the CNN archtecture plays a crucial role in good performance. The final best network contains 16 CONV/FC Layer. Moreover the layers are homogeneous. The filter size of CONV layer and pooling layer is 3X3 and 2X2 respectively for all the corresponding layers. Downside of this model is that huge number of parameters (140M)
- **ResNet**: It features special skip connections and a heavy use of batch normalization. In addition, the architecture have no FC layer at the end of the architecture. The ResNets are the more sophisticated and modern CNN architecture since 2016.

### VGG Net in detail:
The whole VGG Net is composed of CONV layers that perform 3X3 convolutions, stride S=1, padding=1, and pool layers that perform 2X2 max pooling with S=2 and no padding.


- **INPUT:** [224X224X3]    memory: 224*224*3=150K    weights:0

- **CONV3-64:** here '3' means filter size of the layer is 3X3 and 64 means number of filters is 64. Therefore the output size will be  (W-F+2P)/S +1= (224-3+2)/1 +1=224 in each spatial size. and the depth will be 64 which equals to number of filters. therefore the final output volume is 224X224X64. memory=3.2 M , weights=(3*3*3)*64=1728
- **CONV3-64**: 224X224X64, memory:3.2M, weights=(3*3*64)*64=36,864
- **POOL2**: filter size K=2, stride=2. the output size is (W-F)/2 +1 = (224-2)/2 +1= 112 spatial size. therefore the output size is 112X112X64. Memory 800K, weight=0. Notice that the depth after pooling operation remains unchanged and the weights =0 because here no parms needed because pooling operation operates on fixed function.

- **CONV3-128**: oputput: 112X112X128, memory:112*112*128=1.6M, weights=(3*3*64)*128=73,728
- **CONV3-128**: oputput: 112X112X128, memory:112*112*128=1.6M, weights=(3*3*128)*128=147,456
- **POOL2:** output:56X56X128, memory: 56*56*128=400K, weight= 0

- **CONV3-256:** output:56X56X256, memory:800K, weights=(3*3*128)*256=294,912
- **CONV3-256:** output:56X56X256, memory:800K, weights=(3*3*256)*256=589,824
- **CONV3-256:** output:56X56X256, memory:800K, weights=(3*3*256)*256=589,824
- **POOL2:** output: 28X28X256, memory: 200K, weights=0

- **CONV3-512**: Output: 28X28X512, memory:400k, weights=(3*3*256)*512=1,179,648
- **CONV3-512**: Output: 28X28X512, memory:400k, weights=(3*3*512)*512=2,359,296
- **CONV3-512**: Output: 28X28X512, memory:400k, weights=(3*3*256)*512=1,179,648
- **POOL2**:  output:14x14X512, memory:100K, weights=0

- **CONV3-512**: Output: 14X14X512, memory:100k, weights=(3*3*512)*512=2,359,296
- **CONV3-512**: Output: 28X28X512, memory:100k, weights=(3*3*512)*512=2,359,296
- **CONV3-512**: Output: 28X28X512, memory:100k, weights=(3*3*256)*512=1,179,648
- **POOL2**:  output:7x7X512, memory:25K, weights=0


- FC: output: [1X1X4096] memory: 4096 ,weights= 7*7*512*4096=102,760,448
- FC: output: [1X1X4096] memory: 4096, weights= 4096*4096=16,777,216
- FC: output: [1X1X1000] memory: 1000, weights= 4096*1000=4096000


Total Memory:24M*4bytes~=93MB/Image(only forward~*2 for bwd)
Total Params: 138 M parameters

**Most Memory**: In CONV layers

**Most Parameters**: In FC layers




